In [78]:
#================================================
# DATA
#================================================

import duckdb, pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

# creating the conection to the duckdb database file:
con = duckdb.connect("movielens100K.duckdb")

In [79]:
IMDB_DIR = Path("..") / "data" / "Imdb"
IMDB_DIR

WindowsPath('../data/Imdb')

### 1.0 Exploratory  Data Analysis (Enriched Dataset)
#### 1.1 Search for all time know Characters
##### 1.1.1 Ad-hoc Query Characters and actors_name by movieID

In [88]:
movie_id = 165  # <--- coloca aqui o movieId

df = con.execute(f"""
    SELECT
        actor_name,
        characters,
        title
    FROM movielens_actors
    WHERE movieId = {movie_id}
""").df()

df


,actor_name,characters,title
0,Bruce Willis,John McClane,Die Hard: With a Vengeance (1995)
1,Jeremy Irons,Simon,Die Hard: With a Vengeance (1995)
2,Samuel L. Jackson,Zeus,Die Hard: With a Vengeance (1995)
3,Graham Greene,Joe Lambert,Die Hard: With a Vengeance (1995)
4,Colleen Camp,Connie Kowalski,Die Hard: With a Vengeance (1995)
5,Larry Bryggman,Arthur Cobb,Die Hard: With a Vengeance (1995)
6,Anthony Peck,Ricky Walsh,Die Hard: With a Vengeance (1995)
7,Nick Wyman,Targo,Die Hard: With a Vengeance (1995)
8,Sam Phillips,Katya,Die Hard: With a Vengeance (1995)
9,Kevin Chamberlin,Charles Weiss,Die Hard: With a Vengeance (1995)


##### 1.1.2 Bigest Hero Ever: Batman vs Superman

In [81]:
character = "batman"

df = con.execute(f"""
    SELECT
        characters,
        actor_name,
        title AS movie_title
    FROM movielens_actors
    WHERE LOWER(characters) = LOWER('{character}')
    ORDER BY title, actor_name;
""").df()

df


,characters,actor_name,movie_title
0,Batman,Joey Krajcar,American Splendor (2003)
1,Batman,George Clooney,Batman & Robin (1997)
2,Batman,Adam West,Batman (1966)
3,Batman,Michael Keaton,Batman (1989)
4,Batman,Christian Bale,Batman Begins (2005)
5,Batman,Will Friedle,Batman Beyond: Return of the Joker (2000)
6,Batman,Val Kilmer,Batman Forever (1995)
7,Batman,Michael Keaton,Batman Returns (1992)
8,Batman,Ben Affleck,Batman v Superman: Dawn of Justice (2016)
9,Batman,Jason O'Mara,Batman vs. Robin (2015)


In [82]:
character = "superman"

df = con.execute(f"""
    SELECT
        movieId,
        characters,
        actor_name,
        title AS movie_title
    FROM movielens_actors
    WHERE LOWER(characters) LIKE LOWER('%{character}%')
    ORDER BY title, actor_name;
""").df()

df

,movieId,characters,actor_name,movie_title
0,84414,Superman,James Denton,All-Star Superman (2011)
1,6620,Superman,Chris Ambrose,American Splendor (2003)
2,136864,Superman,Henry Cavill,Batman v Superman: Dawn of Justice (2016)
3,27155,Superman,Tim Daly,"Batman/Superman Movie, The (1998)"
4,99813,Superman,Mark Valley,"Batman: The Dark Knight Returns, Part 2 (2013)"
5,5541,Superman,Charlie Sheen,Hot Shots! (1991)
6,122898,Superman,Henry Cavill,Justice League (2017)
7,104419,Superman,Mark Harmon,Justice League: Crisis on Two Earths (2010)
8,102084,Superman,Tim Daly,Justice League: Doom (2012)
9,58404,Superman,Kyle MacLachlan,Justice League: The New Frontier (2008)


##### 1.1.3 Bigest Badass Ever: John Maclane or Rambo

In [89]:
character = "John McClane"

df = con.execute(f"""
    SELECT
        movieId,
        characters,
        actor_name,
        title AS movie_title
    FROM movielens_actors
    WHERE LOWER(characters) LIKE LOWER('%{character}%')
    ORDER BY title, actor_name;
""").df()

df

,movieId,characters,actor_name,movie_title
0,1036,John McClane,Bruce Willis,Die Hard (1988)
1,1370,John McClane,Bruce Willis,Die Hard 2 (1990)
2,165,John McClane,Bruce Willis,Die Hard: With a Vengeance (1995)
3,100498,John McClane,Bruce Willis,"Good Day to Die Hard, A (2013)"
4,53972,John McClane,Bruce Willis,Live Free or Die Hard (2007)


In [90]:
character = "Rambo"

df = con.execute(f"""
    SELECT
        movieId,
        characters,
        actor_name,
        title AS movie_title
    FROM movielens_actors
    WHERE LOWER(characters) LIKE LOWER('%{character}%')
    ORDER BY title, actor_name;
""").df()

df

,movieId,characters,actor_name,movie_title
0,2843,Dadan Karambolo,Srdjan 'Zika' Todorovic,"Black Cat, White Cat (Crna macka, beli macor) ..."
1,2403,Rambo,Sylvester Stallone,First Blood (Rambo: First Blood) (1982)
2,1423,Rambo,Seputla Sebogodi,Hearts and Minds (1996)
3,57528,John Rambo,Sylvester Stallone,Rambo (Rambo 4) (2008)
4,2404,Rambo,Sylvester Stallone,Rambo III (1988)
5,2402,Rambo,Sylvester Stallone,Rambo: First Blood Part II (1985)
6,1571,"Rambo, le chat",Rambo,When the Cat's Away (Chacun cherche son chat) ...
7,5346,Rambo,Antonio Mario Silva Da Silva,Wild Orchid (1990)


##### 1.1.3 Biggest adventure? Indiana Jones or lara Croft?

In [56]:
character = "Indiana Jones"

df = con.execute(f"""
    SELECT
        movieId,
        characters,
        actor_name,
        title AS movie_title
    FROM movielens_actors
    WHERE LOWER(characters) LIKE LOWER('%{character}%')
    ORDER BY title, actor_name;
""").df()

df

,movieId,characters,actor_name,movie_title
0,61348,Indiana Jones,Tony Cox,Disaster Movie (2008)
1,59615,Indiana Jones,Harrison Ford,Indiana Jones and the Kingdom of the Crystal S...
2,1291,Indiana Jones,Harrison Ford,Indiana Jones and the Last Crusade (1989)
3,2115,Indiana Jones,Harrison Ford,Indiana Jones and the Temple of Doom (1984)


In [68]:
character = "Lara Croft"

df = con.execute(f"""
    SELECT
        movieId,
        characters,
        actor_name,
        title AS movie_title
    FROM movielens_actors
    WHERE LOWER(characters) LIKE LOWER('%{character}%')
    ORDER BY title, actor_name;
""").df()

df

,movieId,characters,actor_name,movie_title
0,6564,Lara Croft,Angelina Jolie,Lara Croft Tomb Raider: The Cradle of Life (2003)
1,4367,Lara Croft,Angelina Jolie,Lara Croft: Tomb Raider (2001)
2,184471,Lara Croft,Alicia Vikander,Tomb Raider (2018)


In [60]:
character = "Rambo"

df = con.execute(f"""
    SELECT
        movieId,
        characters,
        actor_name,
        title AS movie_title
    FROM movielens_actors
    WHERE LOWER(characters) LIKE LOWER('%{character}%')
    ORDER BY title, actor_name;
""").df()

df

,movieId,characters,actor_name,movie_title
0,2843,Dadan Karambolo,Srdjan 'Zika' Todorovic,"Black Cat, White Cat (Crna macka, beli macor) ..."
1,2403,Rambo,Sylvester Stallone,First Blood (Rambo: First Blood) (1982)
2,1423,Rambo,Seputla Sebogodi,Hearts and Minds (1996)
3,57528,John Rambo,Sylvester Stallone,Rambo (Rambo 4) (2008)
4,2404,Rambo,Sylvester Stallone,Rambo III (1988)
5,2402,Rambo,Sylvester Stallone,Rambo: First Blood Part II (1985)
6,1571,"Rambo, le chat",Rambo,When the Cat's Away (Chacun cherche son chat) ...
7,5346,Rambo,Antonio Mario Silva Da Silva,Wild Orchid (1990)


##### 1.1.6 And the biggest vilain? Darth Vader, Joker or Hannibal Lecter?

In [61]:
character = "Darth Vader"

df = con.execute(f"""
    SELECT
        movieId,
        characters,
        actor_name,
        title AS movie_title
    FROM movielens_actors
    WHERE LOWER(characters) LIKE LOWER('%{character}%')
    ORDER BY title, actor_name;
""").df()

df

,movieId,characters,actor_name,movie_title
0,127390,Stewie Griffin as Darth Vader,Seth MacFarlane,Family Guy Presents: Blue Harvest (2007)
1,85780,Stewie Griffin as Darth Vader,Seth MacFarlane,Family Guy Presents: It's a Trap (2010)
2,140481,Stewie Griffin as Darth Vader,Seth MacFarlane,"Family Guy Presents: Something, Something, Som..."
3,260,Darth Vader,David Prowse,Star Wars: Episode IV - A New Hope (1977)
4,1196,Darth Vader,David Prowse,Star Wars: Episode V - The Empire Strikes Back...
5,1210,Darth Vader,James Earl Jones,Star Wars: Episode VI - Return of the Jedi (1983)
6,135216,Darth Vader,James Earl Jones,The Star Wars Holiday Special (1978)


In [65]:
character = "Hannibal lecter"

df = con.execute(f"""
    SELECT
        movieId,
        characters,
        actor_name,
        title AS movie_title
    FROM movielens_actors
    WHERE LOWER(characters) LIKE LOWER('%{character}%')
    ORDER BY title, actor_name;
""").df()

df

,movieId,characters,actor_name,movie_title
0,4148,Hannibal Lecter,Anthony Hopkins,Hannibal (2001)
1,5630,Hannibal Lecter,Anthony Hopkins,Red Dragon (2002)
2,593,Dr. Hannibal Lecter,Anthony Hopkins,"Silence of the Lambs, The (1991)"


In [69]:
character = "joker"

df = con.execute(f"""
    SELECT
        movieId,
        characters,
        actor_name,
        title AS movie_title
    FROM movielens_actors
    WHERE LOWER(characters) LIKE LOWER('%{character}%')
    ORDER BY title, actor_name;
""").df()

df

,movieId,characters,actor_name,movie_title
0,26152,The Joker,Cesar Romero,Batman (1966)
1,592,Joker,Jack Nicholson,Batman (1989)
2,27311,The Joker,Mark Hamill,Batman Beyond: Return of the Joker (2000)
3,27155,The Joker,Mark Hamill,"Batman/Superman Movie, The (1998)"
4,113278,Joker,Troy Baker,Batman: Assault on Arkham (2014)
5,99813,Joker,Michael Emerson,"Batman: The Dark Knight Returns, Part 2 (2013)"
6,161354,The Joker,Mark Hamill,Batman: The Killing Joke (2016)
7,79274,Joker,John DiMaggio,Batman: Under the Red Hood (2010)
8,58559,Joker,Heath Ledger,"Dark Knight, The (2008)"
9,4998,John 'Joker' Jackson,Tony Curtis,"Defiant Ones, The (1958)"


#### 1.2 Search for all time know Movies
##### 1.2.1 Best sci fi movies: Star Wars vs star trek vs alien

In [91]:
movie = "Star wars"

df = con.execute(f"""
    SELECT
        movieId,
        title
    FROM movies
    WHERE LOWER(title) LIKE LOWER('%{movie}%')
    ORDER BY movieId, title;
""").df()

df

,movieId,title
0,260,Star Wars: Episode IV - A New Hope (1977)
1,1196,Star Wars: Episode V - The Empire Strikes Back...
2,1210,Star Wars: Episode VI - Return of the Jedi (1983)
3,2628,Star Wars: Episode I - The Phantom Menace (1999)
4,5378,Star Wars: Episode II - Attack of the Clones (...
5,33493,Star Wars: Episode III - Revenge of the Sith (...
6,61160,Star Wars: The Clone Wars (2008)
7,79006,Empire of Dreams: The Story of the 'Star Wars'...
8,122886,Star Wars: Episode VII - The Force Awakens (2015)
9,135216,The Star Wars Holiday Special (1978)


In [92]:
movie = "Star trek"

df = con.execute(f"""
    SELECT
        movieId,
        title
    FROM movies
    WHERE LOWER(title) LIKE LOWER('%{movie}%')
    ORDER BY movieId, title;
""").df()

df

,movieId,title
0,329,Star Trek: Generations (1994)
1,1356,Star Trek: First Contact (1996)
2,1371,Star Trek: The Motion Picture (1979)
3,1372,Star Trek VI: The Undiscovered Country (1991)
4,1373,Star Trek V: The Final Frontier (1989)
5,1374,Star Trek II: The Wrath of Khan (1982)
6,1375,Star Trek III: The Search for Spock (1984)
7,1376,Star Trek IV: The Voyage Home (1986)
8,2393,Star Trek: Insurrection (1998)
9,5944,Star Trek: Nemesis (2002)


#### 1.3 Search for all time know Actors

In [45]:
#Actors who played Batman (any variation, case insensitive)
con.sql("""
SELECT DISTINCT movieId, title, actor_name, characters
FROM movielens_actors
WHERE characters ILIKE '%batman%'
ORDER BY title, actor_name;
""").df()


,movieId,title,actor_name,characters
0,6620,American Splendor (2003),Joey Krajcar,Batman
1,1562,Batman & Robin (1997),George Clooney,Batman
2,26152,Batman (1966),Adam West,Batman
3,592,Batman (1989),Michael Keaton,Batman
4,33794,Batman Begins (2005),Christian Bale,Batman
5,27311,Batman Beyond: Return of the Joker (2000),Will Friedle,Batman
6,153,Batman Forever (1995),Val Kilmer,Batman
7,1377,Batman Returns (1992),Michael Keaton,Batman
8,136864,Batman v Superman: Dawn of Justice (2016),Ben Affleck,Batman
9,131739,Batman vs. Robin (2015),Jason O'Mara,Batman


In [46]:
# Top actors who played Batman (by number of distinct films)
con.sql("""
SELECT
  actor_name,
  COUNT(DISTINCT movieId) AS films_as_batman
FROM movielens_actors
WHERE characters ILIKE '%Batman%'
GROUP BY actor_name
ORDER BY films_as_batman DESC, actor_name;
""").df()


,actor_name,films_as_batman
0,Kevin Conroy,8
1,Ben Affleck,2
2,Michael Keaton,2
3,Peter Weller,2
4,Will Arnett,2
5,Adam West,1
6,Ben McKenzie,1
7,Bruce Greenwood,1
8,Christian Bale,1
9,George Clooney,1


In [47]:
con.sql("SHOW TABLES").df()

,name
0,imdb_map_ids
1,imdb_name_basics
2,imdb_principals
3,imdb_principals_writers
4,imdb_ratings
5,imdb_title_basics
6,imdb_title_crew
7,links
8,movielens_actors
9,movielens_directors


#### Fechar a ligação

In [51]:
con.close()
print("Ligação fechada.")

Ligação fechada.
